# Copyright

<PRE>
Copyright (c) 2023 Bálint Gyires-Tóth - All Rights Reserved
You may use and modify this code for research and development purpuses.
Using this code for educational purposes (self-paced or instructor led) without the permission of the author is prohibited.
</PRE>

# Convolutional Neural Network exercise

In this notebook, you will build a convolutional neural network, which offers superior results over the previous MLP solution. Additionally, you will perform a more detailed evaluation.  

In [2]:
# imports 
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.callbacks import EarlyStopping

# 1. Data
The dataset we will use is the well-known CIFAR10 (Canadian Institute For Advanced Research). Explore the details on the Keras website first: https://keras.io/api/datasets/cifar10/

There isn't much difference between the data preprocessing part and the one we did before. The only difference is that we don't have to reshape images into vectors, but we are working with a 4 dimensional data structure, where the dimensions are: batch, X, Y, and color channels. 

In the event that anything is not clear regarding the data preparation part, please check the previous CIFAR10 notebook.

In [62]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [63]:
# it is in int8 format, the neural network requires float32
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

In [64]:
train_ratio  = 0.8
train_length = X_train.shape[0]
train_split  = int(train_ratio*train_length)
X_valid, Y_valid = X_train[train_split:], Y_train[train_split:]
X_train, Y_train = X_train[:train_split], Y_train[:train_split]

In [65]:
mean = X_train.mean(axis=0)
std  = X_train.std(axis=0)

X_train = (X_train-mean)/std
X_valid = (X_valid-mean)/std
X_test  = (X_test-mean)/std

In [66]:
nb_classes = len(np.unique(Y_train))
Y_train = to_categorical(Y_train, nb_classes)
Y_valid = to_categorical(Y_valid, nb_classes)
Y_test  = to_categorical(Y_test, nb_classes)

In [67]:
print("Shapes of the training, validation and test input data:", X_train.shape, X_valid.shape, X_test.shape)
print("Shapes of the training, validation and test output data:", Y_train.shape, Y_valid.shape, Y_test.shape)
print("Mean values of the training, validation and test input data:", X_train.mean(), X_valid.mean(), X_test.mean())
print("Standard deviation of the training, validation and test input data:", X_train.std(), X_valid.std(), X_test.std())

Shapes of the training, validation and test input data: (40000, 32, 32, 3) (10000, 32, 32, 3) (10000, 32, 32, 3)
Shapes of the training, validation and test output data: (40000, 10) (10000, 10) (10000, 10)
Mean values of the training, validation and test input data: 2.2788842e-09 0.0023437198 0.013331206
Standard deviation of the training, validation and test input data: 1.0000023 0.9965626 0.99783075


# 2. Exercise: training

Create and train a convolutional neural network with the following requirements:
* apply early stopping with patience=5 and monitor the validation accuracy, don't forget to load back the best weights after early stopping
* the number of trainable parameters is lower than 200.000
* the validation accuracy must be higher than 72%

Hints:
* you can always inspect the number of trainable parameters per layer with model.summary()
* usually the most trainable parameters are between the last convolutional layer and the next dense layer -- so you should have a reasonable output size at the last convolutional layer
* regularization helps (e.g. DropOut and/or BatchNormalization)
* use the right activation function at the output layer and the corresponding loss function
* for the chosen activation functions, please use the appropriate initialization method
* ADAM optimizer is a reasonable choice 
* don't forget to turn on GPU support in Colab, otherwise, it would be quite slow to train the networks



In [68]:
## I guess I need to reshape first of all. 
# reshape 3D tensors to 2D tensors
#X_train = X_train.reshape(-1, 32*32*3)
#X_test = X_test.reshape(-1, 32*32*3)
#X_valid = X_valid.reshape(-1, 32*32*3)

In [69]:
es = keras.callbacks.EarlyStopping(patience = 5,
                                   monitor="val_accuraccy",
                                   restore_best_weights=True)
    

# early stopping

In [78]:
# model definition
model = Sequential()
model.add(Conv2D(2,2, activation='relu', kernel_initializer=HeNormal(), input_shape=(32,32,3)))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Flatten())
model.add(Dense(nb_classes, activation='softmax'))


# loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [79]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 31, 31, 2)         26        
                                                                 
 dropout_15 (Dropout)        (None, 31, 31, 2)         0         
                                                                 
 dense_43 (Dense)            (None, 31, 31, 64)        192       
                                                                 
 flatten (Flatten)           (None, 61504)             0         
                                                                 
 dense_44 (Dense)            (None, 10)                615050    
                                                                 
Total params: 615,268
Trainable params: 615,268
Non-trainable params: 0
_________________________________________________________________


Please, check the number of trainable paramers:

In [80]:
print("#trainable parameters:",model.count_params())

#trainable parameters: 615268


In [81]:
# training
network_history = model.fit(X_train, Y_train, 
                            validation_data=(X_valid,Y_valid),
                            batch_size=128,                             
                            epochs=30, 
                            verbose=1, 
                            callbacks=es)


Epoch 1/30
312/313 [============================>.] - ETA: 0s - loss: 1.9171 - accuracy: 0.3344

313/313 [==============================] - 9s 9ms/step - loss: 1.9168 - accuracy: 0.3345 - val_loss: 1.7969 - val_accuracy: 0.3786
Epoch 2/30
312/313 [============================>.] - ETA: 0s - loss: 1.6713 - accuracy: 0.4232

313/313 [==============================] - 3s 8ms/step - loss: 1.6711 - accuracy: 0.4233 - val_loss: 1.7009 - val_accuracy: 0.4147
Epoch 3/30
311/313 [============================>.] - ETA: 0s - loss: 1.5399 - accuracy: 0.4630

313/313 [==============================] - 3s 9ms/step - loss: 1.5398 - accuracy: 0.4631 - val_loss: 1.6382 - val_accuracy: 0.4326
Epoch 4/30
306/313 [============================>.] - ETA: 0s - loss: 1.4550 - accuracy: 0.4927

313/313 [==============================] - 3s 8ms/step - loss: 1.4549 - accuracy: 0.4930 - val_loss: 1.5808 - val_accuracy: 0.4467
Epoch 5/30
307/313 [============================>.] - ETA: 0s - loss: 1.3932 - accuracy: 0.5177

...

313/313 [==============================] - 2s 7ms/step - loss: 0.8740 - accuracy: 0.6936 - val_loss: 1.6167 - val_accuracy: 0.4850
Epoch 27/30
308/313 [============================>.] - ETA: 0s - loss: 0.8624 - accuracy: 0.6985

313/313 [==============================] - 3s 9ms/step - loss: 0.8626 - accuracy: 0.6986 - val_loss: 1.6272 - val_accuracy: 0.4878
Epoch 28/30
308/313 [============================>.] - ETA: 0s - loss: 0.8526 - accuracy: 0.7025

313/313 [==============================] - 2s 7ms/step - loss: 0.8532 - accuracy: 0.7025 - val_loss: 1.6560 - val_accuracy: 0.4790
Epoch 29/30
309/313 [============================>.] - ETA: 0s - loss: 0.8364 - accuracy: 0.7056

313/313 [==============================] - 2s 7ms/step - loss: 0.8363 - accuracy: 0.7053 - val_loss: 1.6517 - val_accuracy: 0.4884
Epoch 30/30
312/313 [============================>.] - ETA: 0s - loss: 0.8259 - accuracy: 0.7120

313/313 [==============================] - 2s 7ms/step - loss: 0.8259 - accuracy: 0.7120 - val_loss: 1.6295 - val_accuracy: 0.4910


# 3. Exercise: evaluation on test data
You will evaluate the classification of the test data using the common metrics and the confusion matrix in this section. 


As a first stes, let's run the predictions and convert the predicted softmax outputs to dense labels. The one-hot encoded labels are also converted back to dense labels, as they are required for the evaluation functions.

In [40]:
probas = model.predict(X_test)
preds  = np.argmax(probas,axis=1)
Y_test_dense = np.argmax(Y_test, axis=1) # get the original dense labels of the test data

313/313 [==============================] - 1s 2ms/step


## 3.1. Exercise: Metrics
please review the common classification metrics (accuracy, precision, recall, F1) for the complete model (not per class). 

Hints:
* use macro averaging
* you have to import the right functions from [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html)


In [ ]:
from sklearn.metrics import <TODO>

In [ ]:
print("Accuracy:",<TODO>)
print("Precision:",<TODO>)
print("Recall:",<TODO>)
print("F1:",<TODO>)


## 3.2. Exercise: Confusion matrix
Calculate the confusion matrix and display the results in a heatmap (with decimal values).

Hints:
* you have to do the imports this time too, e.g. from [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) and the [heatmap function](https://seaborn.pydata.org/generated/seaborn.heatmap.html) of [seaborn](https://seaborn.pydata.org/)
* for displaying the decimal values, you have to turn annot=True and define fmt='d' in the heatmap function of seaborn

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
conf = <TODO>
plt.figure(figsize=(7,6))
sns.heatmap(<TODO>)